In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset=dataset.drop("User ID",axis=1)

In [6]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
independent=dataset[[ 'Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset[['Purchased']]

In [10]:
from sklearn.model_selection  import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test) 

In [12]:
from sklearn.model_selection  import GridSearchCV
from sklearn.svm import SVC

param_grid={'kernel':['rbf','sigmoid','linear','poly'],'C':[10,100,1000,2000,3000],'gamma':['scale','auto']}

grid=GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)


Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['rbf', 'sigmoid', 'linear', 'poly']},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_

grid_predictions = grid.predict(X_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)


In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}: 0.9171245421245421


In [15]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[73  6]
 [ 4 37]]


In [16]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.9654214263661625

In [18]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.037191,0.026097,0.016315,0.004322,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.892857,0.875644,0.841398,0.947015,0.946153,0.900613,0.041029,1
1,0.058890,0.022385,0.021569,0.009152,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.769053,0.733523,0.610390,0.753871,0.759910,0.725349,0.058652,40
2,0.102708,0.091939,0.028646,0.013374,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
3,0.085675,0.042560,0.029189,0.006913,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,12
4,0.078259,0.054693,0.046583,0.026474,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.892857,0.875644,0.841398,0.947015,0.946153,0.900613,0.041029,1
5,0.055553,0.038343,0.019088,0.007855,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.769053,0.733523,0.730543,0.753871,0.759910,0.749380,0.014996,32
6,0.026843,0.007769,0.012299,0.003000,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
7,0.034692,0.013382,0.010793,0.000936,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,12
8,0.041552,0.015754,0.012125,0.003223,100,scale,rbf,"{'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}",0.855314,0.892857,0.859435,0.929144,0.928571,0.893064,0.031996,3
9,0.025042,0.006375,0.016841,0.005365,100,scale,sigmoid,"{'C': 100, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.769053,0.714286,0.750000,0.753871,0.759910,0.749424,0.018709,31
